In [ ]:
import os
from pathlib import Path
import pickle
from fastfusion import Specification
from fastfusion.mapper.FFM.exploration.mapper_multi_einsum import get_sims
from fastfusion.mapper.FFM.joining.simexplore import join_sims
from fastfusion.mapper.FFM.exploration.mapping_filter_tags import get_one_split_tag, get_ffmt_tag


ARCH_DIR          = Path('architecture/')
WORKLOAD_DIR      = Path('workloads/')
MAPPINGS_SIMS_DIR = Path('results/sims/')
MAPPINGS_DATA_DIR = Path('results/data/')


def one_split_tagger(compatibility):
    return get_one_split_tag(compatibility, "MainMemory")


NAME_TO_TAGGER = {
    'one_split': one_split_tagger
}


def get_experiment_name(tagger_name, arch_name: list[str], workload_name):
    combined_arch_name = '+'.join(arch_name)
    return f'matmul8_mixed.{tagger_name}.{combined_arch_name}.{workload_name}'
    return f'matmul8_mixed.{tagger_name}.{combined_arch_name}.{workload_name}'


def get_sims_with_cache(tagger_name=None,
                        refresh_cache=False,
                        arch_name: list[str]=['snowcat'],
                        workload_name='matmuls8_mixed'):
    data_name  = get_experiment_name(tagger_name, arch_name, workload_name)
    result_pickle_name = MAPPINGS_DATA_DIR / f'{data_name}.pkl'
    if result_pickle_name.is_file() and not refresh_cache:
        with open(result_pickle_name, 'rb') as f:
            mappings = pickle.load(f)
            print(f'Loaded final results from cache {result_pickle_name}')
            return mappings

    if tagger_name is None:
        tagger = None
    else:
        tagger = NAME_TO_TAGGER[tagger_name]

    all_sims = []
    for a in arch_name:
        sims_name = get_experiment_name(tagger_name, [a], workload_name)
        sims_pickle_name = MAPPINGS_SIMS_DIR / f'{sims_name}.pkl'
        if sims_pickle_name.is_file() and not refresh_cache:
            with open(sims_pickle_name, 'rb') as f:
                sims = pickle.load(f)
                print(f'Loaded SIMs from {sims_pickle_name}')
                print(f'Loaded SIMs from {sims_pickle_name}')
                all_sims.append(sims)
                continue

        spec = Specification.from_yaml(
            ARCH_DIR / f'{a}.arch.yaml',
            WORKLOAD_DIR / f'{workload_name}.workload.yaml'
        )
        spec.calculate_component_energy_area()
        workload = spec.workload
        renames = spec.renames
        workload = spec.workload
        renames = spec.renames
        flattened_architecture = spec.get_flattened_architecture()
        sims, decompress_data = get_sims(spec, flattened_architecture, tagger=tagger)

        os.makedirs(MAPPINGS_SIMS_DIR, exist_ok=True)
        with open(sims_pickle_name, 'wb') as f:
            pickle.dump(sims, f)

        all_sims.append(sims)

    combined_sims = {}
    for sims in all_sims:
        for einsum, sims_for_einsum in sims.items():
            if einsum not in combined_sims:
                combined_sims[einsum] = []
            combined_sims[einsum].extend(sims_for_einsum)

    spec = Specification.from_yaml(
        ARCH_DIR / f'{a}.arch.yaml',
        WORKLOAD_DIR / f'{workload_name}.workload.yaml'
    )
    flattened_architecture = spec.get_flattened_architecture()
    mappings = join_sims(combined_sims, spec, flattened_architecture, drop_valid_reservations=False)
    mappings.decompress(decompress_data)

    with open(result_pickle_name, 'wb') as f:
        pickle.dump(mappings, f)
        print(f'Saved results to cache {result_pickle_name}')

    return mappings

mappings_looptree = get_sims_with_cache('one_split')
mappings_full = get_sims_with_cache()
mappings_tileflow = get_sims_with_cache(tagger_name='one_split', arch_name=['snowcat_even'])
mappings_ffmt = get_sims_with_cache(tagger_name='one_split', arch_name=['snowcat_ffmt'])

INFO        Loading yaml file architecture/snowcat.arch.yaml
INFO        Found top key variables in architecture/snowcat.arch.yaml
INFO        Found top key architecture in architecture/snowcat.arch.yaml
INFO        Loading yaml file workloads/matmuls8_mixed.workload.yaml
INFO        Found top key workload in workloads/matmuls8_mixed.workload.yaml
WARNING     Loading configuration file from /root/.config/fastfusion/config.yaml
INFO        Calculated "0.5" = 0.5.
Generating storage and loop choices for Einsum Matmul1: 20it [00:00, 1645.99it/s]


choice T0 in GlobalBuffer
T0 in GlobalBuffer
T1 in GlobalBuffer
W0 in GlobalBuffer
choice T1 in GlobalBuffer
T1 in GlobalBuffer
W0 in GlobalBuffer
choice W0 in GlobalBuffer
W0 in GlobalBuffer
choice W0 in GlobalBuffer
T1 in GlobalBuffer
W0 in GlobalBuffer
choice T1 in GlobalBuffer
T1 in GlobalBuffer
choice T1 in GlobalBuffer
T0 in GlobalBuffer
T1 in GlobalBuffer
W0 in GlobalBuffer
choice T0 in GlobalBuffer
T0 in GlobalBuffer
W0 in GlobalBuffer
choice W0 in GlobalBuffer
W0 in GlobalBuffer
choice W0 in GlobalBuffer
T0 in GlobalBuffer
W0 in GlobalBuffer
choice T0 in GlobalBuffer
T0 in GlobalBuffer
choice W0 in GlobalBuffer
T0 in GlobalBuffer
T1 in GlobalBuffer
W0 in GlobalBuffer
choice T0 in GlobalBuffer
T0 in GlobalBuffer
T1 in GlobalBuffer
choice T1 in GlobalBuffer
T1 in GlobalBuffer
choice T1 in GlobalBuffer
T0 in GlobalBuffer
T1 in GlobalBuffer
choice T0 in GlobalBuffer
T0 in GlobalBuffer
choice T0 in GlobalBuffer
T0 in GlobalBuffer
T1 in GlobalBuffer
W0 in GlobalBuffer
choice T1 in G

Generating storage and loop choices for Einsum Matmul2: 18it [00:00, 1551.50it/s]


choice T1 in GlobalBuffer
T1 in GlobalBuffer
T2 in GlobalBuffer
W1 in GlobalBuffer
choice T2 in GlobalBuffer
T2 in GlobalBuffer
W1 in GlobalBuffer
choice W1 in GlobalBuffer
W1 in GlobalBuffer
choice W1 in GlobalBuffer
T2 in GlobalBuffer
W1 in GlobalBuffer
choice T2 in GlobalBuffer
T2 in GlobalBuffer
choice T2 in GlobalBuffer
T1 in GlobalBuffer
T2 in GlobalBuffer
W1 in GlobalBuffer
choice T1 in GlobalBuffer
T1 in GlobalBuffer
W1 in GlobalBuffer
choice W1 in GlobalBuffer
W1 in GlobalBuffer
choice W1 in GlobalBuffer
T1 in GlobalBuffer
W1 in GlobalBuffer
choice T1 in GlobalBuffer
T1 in GlobalBuffer
choice W1 in GlobalBuffer
T1 in GlobalBuffer
T2 in GlobalBuffer
W1 in GlobalBuffer
choice T1 in GlobalBuffer
T1 in GlobalBuffer
T2 in GlobalBuffer
choice T2 in GlobalBuffer
T2 in GlobalBuffer
choice T2 in GlobalBuffer
T1 in GlobalBuffer
T2 in GlobalBuffer
choice T1 in GlobalBuffer
T1 in GlobalBuffer
choice T1 in GlobalBuffer
T1 in GlobalBuffer
T2 in GlobalBuffer
W1 in GlobalBuffer
choice T2 in G

Grouping Partial Mappings for Matmul2: 100%|██████████| 99/99 [00:00<00:00, 7029.08it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'results/sims/matmul8_mixed.one_split.snowcat.matmuls8_mixed.pkl'

In [7]:
from fastfusion.frontend import Workload
from fastfusion.visualization.interactive import plotly_show

workload = Workload.from_yaml('workloads/matmuls8_mixed.workload.yaml')

plotly_show(mappings_full.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=workload.einsum_names)

INFO        Loading yaml file workloads/matmuls8_mixed.workload.yaml
INFO        Found top key workload in workloads/matmuls8_mixed.workload.yaml
WARNING     Trying to parse a single element dictionary as a Workload. 


INFO        Loading yaml file workloads/matmuls8_mixed.workload.yaml
INFO        Found top key workload in workloads/matmuls8_mixed.workload.yaml
WARNING     Trying to parse a single element dictionary as a Workload. 


    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…

In [ ]:
plotly_show(mappings_looptree.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=workload.einsum_names)

In [ ]:
plotly_show(mappings_looptree.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=workload.einsum_names)

In [3]:

plotly_show(mappings_tileflow.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=workload.einsum_names)

    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…

In [4]:
plotly_show(mappings_ffmt.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=workload.einsum_names)

    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…